<img src="ETSE.png" align="center" width="6800">
<br><br><br>
<font color="#CA3532"><h1 align="left">Diagramas de Influencia.</h1></font>
<font color="#6E6E6E"><h3 align="left">Modelos gráficos probabilísticos. Máster Oficial Ciencia Datos. UV.</h2></font> 

Antonio J. Serrano 

https://agrum.gitlab.io/pages/pyagrum.html

### Entorno MGP con conda (alternativa 1)

conda create -n MGP python=3.9

conda activate MGP 

conda install -c conda-forge pyagrum


(en windows, seguramente hará falta https://graphviz.org/download/)

### Entorno MGP con pip (alternativa 2)
pip install pyagrum

(instalar dependencias... pip install numpy scipy matplotlib pydotplus)

### Utilizar Google Colab (alternativa 3)

!pip install pyagrum

In [2]:
import numpy as np
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

gum.config["influenceDiagram","utility_show_stdev"]=False
#gum.config["influenceDiagram","utility_shape"] = "diamond"


# Ejemplo Médico 0. Sin posibilidad de acción.

Si no hay posibilidad de dar antibiótico, entonces asumimos que es equivalente a la acción no dar antibiótico. Calculamos el valor esperado de la utilidad.

In [3]:
prevalence=0.14
U_X=[10,3]
MEU=U_X[0]*(1-prevalence)+U_X[1]*prevalence
MEU #valor esperado medio de la utilidad    

9.02

# Ejemplo Médico 1. Decisiones en completa Incertidumbre (con riesgo y sin información).

¿Es mejor dar antibiótico o no? Tenemos que tomar la decisión en base únicamente a la prevalencia de la enfermedad $P(X)$. ¿Por qué no hay un arco entre $X$ y $D$?

<img src="EjemploMedico2.png" width="50%">

Ejemplo de creación del diagrama de influencia.

"X->$U<-*D"

"X{+x|-x}->$U<-*D{+d|-d}" si queremos dar etiquetas a las variables

**Grafo/dependencias**

-   Añadimos relaciones con ->

-   Con ; añadimos más dependencias

**Tipos Nodos**

-   Probabilísticos (redondos). Tendremos que especificar DI.cpt("X")

-   Decisión (cuadrados) con el símbolo *

-   Utilidad (rombos) con el símbolo $. Tendremos que especificar DI.utility("U")

In [ ]:
#DI=gum.fastID("X{+x|-x}->$U<-*D{+d|-d}")
DI=gum.fastID("X->$U<-*D")
#print(DI)
gnb.showInfluenceDiagram(DI) #muestra el diagrama

Establecemos la probabilidad de la enfermedad $P(X)$ con DI.cpt("X").fillWith()

In [ ]:
prevalence=0.14
#solucion 


Establecemos la utilidad de la enfermedad $U(X,D)$ con DI.utility("U").fillWith()

In [ ]:
#   +x=1 -x=0
# +d=1  8  9
# -d=0  3 10
# DI.utility("U")[0,0,:]=10
# DI.utility("U")[0,1,:]=3
# DI.utility("U")[1,0,:]=9
# DI.utility("U")[1,1,:]=8
U_DX=[10,3,9,8]
#solucion


Utilzamos LIMID para resolver el diagrama de influencia, los resultados son los mismos que con el diagrama de influencia clasico.

_This inference considers the provided model as a LIMID rather than an influence diagram. It is an optimized implementation of the LIMID resolution algorithm. However an inference on a classical influence diagram can be performed by adding a assumption of the existence of the sequence of decision nodes to be solved, which also implies that the decision choices can have an impact on the rest of the sequence._

In [ ]:
ie=gum.ShaferShenoyLIMIDInference(DI)
print(([list(DI.names())[i] for i in DI.topologicalOrder()]))
gnb.flow.row(ie.reducedLIMID(),ie.reducedGraph(),ie.junctionTree())

Inferenciamos (aunque no tenemos ningua evidencia).

In [ ]:
ie.makeInference() #inplace

Analizamos los resultados. Decisión óptima... 

In [ ]:
ie.optimalDecision("D")

Valor esperado de la utilidad de cada Decisión. 

In [ ]:
ie.posteriorUtility("D")

También podemos estudiar los nodos probabilísticos. 

In [ ]:
ie.posterior("X")

Finalmente el valor esperado medio de la utilidad. A este valor se le denomina Expected Value Without Information (EVwI) o similar.

In [ ]:
#EVwPI
#solución


Visualmente... 

In [ ]:
gnb.flow.row(ie.posterior("D"),
             ie.optimalDecision("D"),
             ie.posteriorUtility("D"),
             captions=['P(D|X)','Decisión Óptima','Valor de la decisión'])

Con estos datos podemos representar el diagrama de influencia resuelto. Utilizamos la función row de la libreria pyAgrum `gnb.flow.row(,,,)`

In [ ]:
gnb.flow.row(DI,gnb.getInference(DI)) 

# Ejemplo Médico 2. Decisiones con información perfecta (sin riesgo). 

¿Qué pasaría si fuéramos adivinos? Es decir, si dispusiéramos de la información perfecta. 

<img src="EjemploMedico1.png" width="50%">

Crea el nuevo diagrama. ¿Cuál es la decisión óptima? ¿Es siempre la misma? ¿Con qué valor? 

In [ ]:
#solución


Interpreta el diagrama de influencia resuelto. 

In [ ]:
gnb.flow.row(DI,gnb.getInference(DI)) 

Valor esperado de disponer de la información perfecta (EV|PI) 

In [ ]:
#EV_PI
#solucion


Expected Value of Perfect Information (EVPI)=EVPI=EV_PI-EVwI. Interpretación del EVPI
- Si EVPI es alto → La incertidumbre tiene un gran impacto en la decisión y obtener información adicional es muy valioso.
- Si EVPI es bajo → La mejor decisión es relativamente clara incluso sin información perfecta.
- Si EVPI es cero → Tener información adicional no cambia la decisión óptima.

In [ ]:
#EVPI
#solucion 


# Ejemplo Médico 3. Decisiones con riesgo pero aportando información (normalmente por estimación). 

Hemos visto que disponer de información es bueno. ¿cómo la introducimos? Con un estimador (ej. modelo de aprendizaje máquina). 

<img src="EjemploMedico3.png" width="50%">

Crea el diagrama de influencia, X y U. 

In [ ]:
#solucion


Introduce la CPT del nodo Y sabiendo que la prueba tiene una Sensibilidad de 0.91 y un Especificidad de 0.97. 

In [ ]:
se=0.91
sp=0.97
#solución


Resuelve e interpreta el diagrama de influencia.

In [ ]:
ie=gum.ShaferShenoyLIMIDInference(DI)
ie.makeInference()
gnb.flow.row(DI,gnb.getInference(DI)) 

Determina la probabilidad de Y=1 sabiendo que X=0. Para introducir evicencias se utiliza el método setEvidence ie.setEvidence({'X':0}). ¿Cuál es el valor esperado? ¿Cómo puede ser mayor que el EV_PI?

In [ ]:
#solución


Observa cómo cambia la decisión óptima cuando Y=1 y Y=0.

In [ ]:
#solución


# Ejemplo Médico 4. Decisiones con riesgo pero aportando información y pagando por ella (coste de la estimación).

Nota: 

Para indicar que no hay estimación de X a partir de Y se puede realizar de dos maneras:
* Añadiendo una salida más a Y (ej. nr) para indicar que no hay respuesta [preferible]
* Utilizando una distribución de probabilidad no informativa (en este caso 1/2) para que D no tenga información de X (el nodo Y marea X para que D no tenga información privilegiada)

El arco de predecencia T->D no aporta nada en este ejercicio porque D recibe la información a través de Y

In [ ]:
DI=gum.fastID("X{+x|-x}->$U<-*D{+d|-d};X->Y{+y|-y|nr}->*D;Y<-*T{+t|-t}->$U;*T->*D")
#DI=gum.fastID("X{+x|-x}->$U<-*D{+d|-d};X->Y{+y|-y|nr}->*D;Y<-*T{+t|-t}->$U")
#DI=gum.fastID("X[2]->$U<-*D[2];X->Y[3]->*D;Y<-*T[2]->$U;*T->*D")
gnb.showInfluenceDiagram(DI)
#solución


Valor Esperado de Test

Valor Esperado de Test y Resultado de la prueba Y